In [4]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

In [5]:
data = pd.read_csv("pre_validate.csv", parse_dates=["date"])
data.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,date,store_nbr,family,id,sales,onpromotion,transactions,oil,city,state,type,cluster,work_day,nat_terremoto,nat_navidad,nat_dia la madre,nat_dia trabajo,nat_primer dia ano,nat_futbol,nat_dia difuntos,day,month,year,day_of_week,day_of_year,week_of_year,date_index,nat_batalla_de_pichincha,nat_carnaval,nat_cyber_monday,nat_independencia_de_cuenca,nat_independencia_de_guayaquil,n_viernes_santo,day_of_month,week_of_month,is_wknd,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,season,wageday,day_to_nearest_holiday,day_from_nearest_holiday
0,0,0,0,2013-01-01,store_nbr_1,AUTOMOTIVE,0.0,NaN,NaN,0.0,93.14,city_quito,state_pichincha,type_D,cluster_13,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,2013,1,1,1,0,0,0,0,0,0,0,1,1,0,1,1,0,1,0,1,0,0,0,0,0
1,1,1,1,2013-01-01,store_nbr_1,BABY CARE,1.0,NaN,NaN,0.0,93.14,city_quito,state_pichincha,type_D,cluster_13,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,2013,1,1,1,0,0,0,0,0,0,0,1,1,0,1,1,0,1,0,1,0,0,0,0,0
2,2,2,2,2013-01-01,store_nbr_1,BEAUTY,2.0,NaN,NaN,0.0,93.14,city_quito,state_pichincha,type_D,cluster_13,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,2013,1,1,1,0,0,0,0,0,0,0,1,1,0,1,1,0,1,0,1,0,0,0,0,0
3,3,3,3,2013-01-01,store_nbr_1,BEVERAGES,3.0,NaN,NaN,0.0,93.14,city_quito,state_pichincha,type_D,cluster_13,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,2013,1,1,1,0,0,0,0,0,0,0,1,1,0,1,1,0,1,0,1,0,0,0,0,0
4,4,4,4,2013-01-01,store_nbr_1,BOOKS,4.0,NaN,NaN,0.0,93.14,city_quito,state_pichincha,type_D,cluster_13,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,2013,1,1,1,0,0,0,0,0,0,0,1,1,0,1,1,0,1,0,1,0,0,0,0,0


In [6]:
from darts_helper.preprocess import get_covariates, get_target_series


# static covariates
static_cols = ["city", "state", "type", "cluster"]
target_dict, pipe_dict, id_dict = get_target_series(data, static_cols)

selected_holidays = [
    "nat_terremoto", "nat_navidad", "nat_dia la madre", "nat_dia trabajo",
    "nat_primer dia ano", "nat_futbol", "nat_dia difuntos", "nat_batalla_de_pichincha",
    "nat_carnaval", 'nat_cyber_monday', 'nat_independencia_de_cuenca',
    'nat_independencia_de_guayaquil', 'n_viernes_santo']

time_based_to_add = ['day_of_month', 'week_of_month', 'is_wknd', 'quarter', 'is_month_start',
       'is_month_end', 'is_quarter_start', 'is_quarter_end', 'is_year_start',
       'is_year_end', 'season', 'wageday', 'day_to_nearest_holiday', 'day_from_nearest_holiday',
       "day", "month", "year", "day_of_week", "day_of_year", "week_of_year", "date_index", "work_day"]

# past covariates
past_cols = ["transactions"]

# future covariates
future_cols = [
    "oil", "onpromotion", *selected_holidays, *time_based_to_add
]

# future covariates from computing the moving averages
future_ma_cols = ["oil", "onpromotion"]

past_dict, future_dict = get_covariates(data, past_cols, future_cols, future_ma_cols)

Extracting covariates: 100%|██████████| 33/33 [02:26<00:00,  4.42s/it]


In [7]:
from darts_helper.trainer import Trainer

TRAINER_CONFIG = {
    "target_dict": target_dict,
    "pipe_dict": pipe_dict,
    "id_dict": id_dict,
    "past_dict": past_dict,
    "future_dict": future_dict,
    
    # time series cross-validation using a rolling forecasting origin
    "forecast_horizon": 16, # the length of the validation set
    "folds": 1, # the number of training sets (setting to 1 means the standard train-validation split)
    
    # the number of previous days to check for zero sales; if all are zero, generate zero forecasts
    "zero_fc_window": 15,
    
    "static_covs": "keep_all",
    "past_covs": "keep_all",
    "future_covs": "keep_all",
    "models": []
}

trainer = Trainer(**TRAINER_CONFIG)

In [8]:
BASE_CONFIG = {
    "random_state": 0,
    
    # the number of lag values of the target series
    "lags": 63,
    
    # the number of lag values of the past covariates
    "lags_past_covariates": list(range(-16, -23, -1)),
    
    # the number of (past, future-1) lag values of the future covariates
    "lags_future_covariates": (14, 1),
    
    # the number of days ahead that the model is forecasting given today's input data
    "output_chunk_length": 1,
}

trainer.train(["lr"], [BASE_CONFIG], drop_before="2015-01-01")

Performing validation:   0%|          | 0/33 [00:00<?, ?it/s]

Performing validation:   3%|▎         | 1/33 [00:09<05:19,  9.99s/it]

AUTOMOTIVE                 | lr: 0.50098


Performing validation:   6%|▌         | 2/33 [00:17<04:26,  8.59s/it]

BABY CARE                  | lr: 0.18491


...

Performing validation:  97%|█████████▋| 32/33 [03:56<00:04,  4.38s/it]

SCHOOL AND OFFICE SUPPLIES | lr: 0.67460


Performing validation: 100%|██████████| 33/33 [04:00<00:00,  7.30s/it]

SEAFOOD                    | lr: 0.45124
Average RMSLE | lr: 0.37671


: 